In [ ]:
# https://github.com/marqo-ai/marqo
# https://marqo.pages.dev/End-to-End%20Examples/streamlit_demo/

In [ ]:
# This is for the marqo docker container to read files from local os. For more info on this please visit this link.
# https://github.com/marqo-ai/marqo/issues/35
# python3 -m http.server 8222

In [ ]:
# sudo chmod 666 /var/run/docker.sock
# sudo docker rm -f marqo;docker run --name marqo -it --privileged -p 8882:8882 --add-host host.docker.internal:host-gateway marqoai/marqo:0.0.3

In [ ]:
%%capture
!pip install wandb
!pip install marqo

In [ ]:
import wandb
wandb.login()

In [ ]:
project_name = "krea-open-prompts"
run_name = "insert-open-prompts-sd-marqo"
tags = ["download", "stable_diffusion", "insert", "marqo"]
_config = {
    # option of "prompts" or "sample_prompts"
    "dataset": "sample_prompts"
}

In [ ]:
run = wandb.init(project=project_name, name=run_name, tags=tags, config=_config)

In [ ]:
config = run.config
dataset_name = config['dataset']

In [ ]:
dataset_path = run.use_artifact(f'{dataset_name}:latest', type='processed_data').download()

In [ ]:
from pathlib import Path

In [ ]:
df_path = Path(dataset_path, f"{dataset_name}.csv")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(df_path)

In [ ]:
#TODO: better way to prepend artifact path
df["local_image_location"] = str(dataset_path) + "/" + dataset_name + "/" + df["local_image_location"]

In [ ]:
#Additional prepend for marqo
df["local_image_location"] = "http://host.docker.internal:8222/" + df["local_image_location"]

In [ ]:
df["local_image_location"].iloc[0]

In [ ]:
data_shape = df.shape

In [ ]:
data = df.to_dict('records')

In [ ]:
import gc

In [ ]:
del df
gc.collect()

In [ ]:
from marqo import Client
from marqo.errors import MarqoApiError

In [ ]:
marqo_settings = {
    "index_defaults": {
        "treat_urls_and_pointers_as_images": True,
        "image_preprocessing": {
            "patch_method": "frcnn"
        },
        "model":"ViT-B/16",
        "normalize_embeddings":True,
    },
}

In [ ]:
mq = Client()

In [ ]:
index_name = "stable-diffusion-open-prompts"

In [ ]:
try:
    mq.create_index(index_name, settings_dict=marqo_settings)
    print("Risen from nothing, it wishes to learn")
except:
    mq.index(index_name).delete()
    mq.create_index(index_name, settings_dict=marqo_settings)
    print("It has been killed and brought back anew sire. Thirsty for knowledge")

In [ ]:
from tqdm import tqdm
from multiprocessing import cpu_count
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == "cuda":
    NUM_PROCS = torch.cuda.device_count()
else:
    NUM_PROCS = cpu_count()

In [ ]:
num_batches = 1000
batch_size = data_shape//num_batches

In [ ]:
# here we use parallel indexing to speed up the task
responses = mq.index(index_name).add_documents(data, device=device, processes=NUM_PROCS, batch_size=batch_size)
# for data_doc in tqdm(data):
#     try:
#         responses = mq.index(index_name).add_documents([data_doc], device=device)
#         # print(f"<SUCCESS>\nAdded prompt:\n{data_doc['prompt']}\nURI: {data_doc['local_image_location']}\n")
#     except:
#         print(f"<FAILURE>\nSkipping prompt:\n{data_doc['prompt']}\nURI: {data_doc['local_image_location']}\n")

In [ ]:
#TODO: Split into two notebooks
# First is half of this -> prep data for insertion
# That includes the transforms done here
# instead of ignoring the problematic opened images, let us pre-download all the images and place it alongside tha appropriate dataset
#     and have the image location be the name of the file where it will prepended with the artifact location after downlaoded
# If it is rate limit add some logic to check the files in an artifact against the table of downloaded files already
# and download that which was not downloaded and relog with that which was alrewady logged?
# Second will be the insertion which will simply pull the appropriate data down and log